In [86]:
import pandas as pd
import numpy as np
import json
from pathlib import Path

LIST_URL: str = 'https://github.com/1Hive/apiary/raw/master/data/app_mining_cycle1/organisation_scores.csv'
ORGS_PATH: Path = Path('../../datawarehouse/aragon/organizations.arr')

In [87]:
df = pd.read_csv(LIST_URL)
df = df[['ADDRESS', 'ENS_NAME']]
df['ADDRESS'] = df['ADDRESS'].str.lower()
print("Duplicates:", df.duplicated('ADDRESS').any())
df

Duplicates: False


,ADDRESS,ENS_NAME
0,0x3ecd508f87ef27417474b0fdf8dd90a3b2b337c9,NaN
1,0xbc435a218374859ca03f675f7697fd7e7ce805cb,trust.aragonid.eth
2,0x49c8c959ce8692978e4f3b169ffd179ec241adaf,budget.aragonid.eth
3,0x40204daacb1480019a7a6826c699903df94ee019,network.aragonid.eth
4,0x635193983512c621e6a3e15ee1dbf36f0c0db8e0,a1.aragonid.eth
...,...,...
1449,0x37a672a49225b640a7f2c8a77a7a8aab8b845e2a,NaN
1450,0x62af9d43a2a22f61e8800a485156d9a8abbe9e84,decentralrecords.aragonid.eth
1451,0xd8cfbb517ae055a4d724ec3b9cd732e9e047d835,tedao.aragonid.eth
1452,0x1b86593fad0cb3a19efec096c748cfc6fcdcf7a3,tokenengineering.aragonid.eth


In [88]:
if ORGS_PATH.is_file():
    dfo = pd.read_feather(ORGS_PATH)
else:
    dfo = pd.DataFrame(columns=['network', 'orgAddress', 'name'])
dfo = dfo[dfo['network'] == 'mainnet']
dfo = dfo[['orgAddress', 'name']]
print("Duplicates:", dfo.duplicated('orgAddress').any())
dfo

Duplicates: False


,orgAddress,name


In [89]:
dfm = dfo.merge(df, left_on='orgAddress', right_on='ADDRESS').drop(columns=['ADDRESS'])
dfm = dfm.dropna(subset='ENS_NAME')
print("Duplicates:", dfm.duplicated('orgAddress').any())
dfm

Duplicates: False


,orgAddress,name,ENS_NAME


In [90]:
dfo[dfo.duplicated('orgAddress', keep=False)].sort_values('orgAddress')

,orgAddress,name


In [91]:
!git checkout 1e7a641ee8d97e7fbe52bf5ff2753bc3183398cd -- ../../cache_scripts/aragon/dao_names.json

with open('../../cache_scripts/aragon/dao_names.json', 'r') as jf:
    dao_names = json.load(jf)
dao_names

{'mainnet': [{'address': '0xF47917B108ca4B820CCEA2587546fbB9f7564b56',
   'domain': 'dcl.eth',
   'name': 'Decentraland'},
  {'address': '0xfe1f2de598f42ce67bb9aad5ad473f0272d09b74',
   'domain': 'meloncouncil.eth',
   'name': 'Melon Council'},
  {'address': '0x2de83b50af29678774d5abc4a7cb2a588762f28c',
   'domain': 'governance.aragonproject.eth',
   'name': 'Aragon Governance'},
  {'address': '0x635193983512c621e6a3e15ee1dbf36f0c0db8e0',
   'domain': 'a1.aragonid.eth',
   'name': 'Aragon One'},
  {'address': '0x67757a18eda83125270ef94dcec7658eb39bd8a5',
   'domain': 'blankdao.aragonid.eth',
   'name': 'BlankDAO'},
  {'address': '0xcd3d9b832bff15e0a519610372c6aac651872dde',
   'domain': '',
   'name': 'MyBit'},
  {'address': '0x0ee165029b09d91a54687041adbc705f6376c67f',
   'domain': '',
   'name': 'Livepeer'},
  {'address': '0x5aad137d8f7d2dc6e1b2548c059b1483360bcc6a',
   'domain': 'brightid.aragonid.eth',
   'name': 'BrightID'},
  {'address': '0x5fEED010a99f695852F8eB7B12E77CF6eCd7bE1

In [92]:
dfn = pd.DataFrame(dao_names['mainnet'])
dfn = dfn[dfn['source'] != LIST_URL]
dfn['address'] = dfn['address'].str.lower()
print("Duplicates:", dfn.duplicated('address').any())
dfn

Duplicates: False


,address,domain,name,source
0,0xf47917b108ca4b820ccea2587546fbb9f7564b56,dcl.eth,Decentraland,NaN
1,0xfe1f2de598f42ce67bb9aad5ad473f0272d09b74,meloncouncil.eth,Melon Council,NaN
2,0x2de83b50af29678774d5abc4a7cb2a588762f28c,governance.aragonproject.eth,Aragon Governance,NaN
3,0x635193983512c621e6a3e15ee1dbf36f0c0db8e0,a1.aragonid.eth,Aragon One,NaN
4,0x67757a18eda83125270ef94dcec7658eb39bd8a5,blankdao.aragonid.eth,BlankDAO,NaN
5,0xcd3d9b832bff15e0a519610372c6aac651872dde,,MyBit,NaN
6,0x0ee165029b09d91a54687041adbc705f6376c67f,,Livepeer,NaN
7,0x5aad137d8f7d2dc6e1b2548c059b1483360bcc6a,brightid.aragonid.eth,BrightID,NaN
8,0x5feed010a99f695852f8eb7b12e77cf6ecd7be17,lightwave.aragonid.eth,Lightwave,NaN
9,0x40204daacb1480019a7a6826c699903df94ee019,network.aragonid.eth,Aragon Network,NaN


In [93]:
dfn[dfn.duplicated('address', keep=False)].sort_values('address')

,address,domain,name,source


In [94]:
dfn_new = pd.DataFrame(columns=dfn.columns)
dfn_new.address = dfm.orgAddress
dfn_new.domain = dfm.ENS_NAME
dfn_new.source = LIST_URL
dfn_new

,address,domain,name,source


In [95]:
dfn_concat = pd.concat([dfn, dfn_new])
dfn_concat = dfn_concat.replace(np.NaN, None)
dfn_concat = dfn_concat.drop_duplicates('address')
print("Duplicates:", dfn_concat.duplicated('address').any())
dfn_concat

Duplicates: False


,address,domain,name,source
0,0xf47917b108ca4b820ccea2587546fbb9f7564b56,dcl.eth,Decentraland,None
1,0xfe1f2de598f42ce67bb9aad5ad473f0272d09b74,meloncouncil.eth,Melon Council,None
2,0x2de83b50af29678774d5abc4a7cb2a588762f28c,governance.aragonproject.eth,Aragon Governance,None
3,0x635193983512c621e6a3e15ee1dbf36f0c0db8e0,a1.aragonid.eth,Aragon One,None
4,0x67757a18eda83125270ef94dcec7658eb39bd8a5,blankdao.aragonid.eth,BlankDAO,None
5,0xcd3d9b832bff15e0a519610372c6aac651872dde,,MyBit,None
6,0x0ee165029b09d91a54687041adbc705f6376c67f,,Livepeer,None
7,0x5aad137d8f7d2dc6e1b2548c059b1483360bcc6a,brightid.aragonid.eth,BrightID,None
8,0x5feed010a99f695852f8eb7b12e77cf6ecd7be17,lightwave.aragonid.eth,Lightwave,None
9,0x40204daacb1480019a7a6826c699903df94ee019,network.aragonid.eth,Aragon Network,None


In [96]:
dfn_concat2 = dfn_concat.copy()
dfn_concat2['name'] = dfn_concat2['name'].fillna(dfn_concat2['domain'])
dfn_concat2

,address,domain,name,source
0,0xf47917b108ca4b820ccea2587546fbb9f7564b56,dcl.eth,Decentraland,None
1,0xfe1f2de598f42ce67bb9aad5ad473f0272d09b74,meloncouncil.eth,Melon Council,None
2,0x2de83b50af29678774d5abc4a7cb2a588762f28c,governance.aragonproject.eth,Aragon Governance,None
3,0x635193983512c621e6a3e15ee1dbf36f0c0db8e0,a1.aragonid.eth,Aragon One,None
4,0x67757a18eda83125270ef94dcec7658eb39bd8a5,blankdao.aragonid.eth,BlankDAO,None
5,0xcd3d9b832bff15e0a519610372c6aac651872dde,,MyBit,None
6,0x0ee165029b09d91a54687041adbc705f6376c67f,,Livepeer,None
7,0x5aad137d8f7d2dc6e1b2548c059b1483360bcc6a,brightid.aragonid.eth,BrightID,None
8,0x5feed010a99f695852f8eb7b12e77cf6ecd7be17,lightwave.aragonid.eth,Lightwave,None
9,0x40204daacb1480019a7a6826c699903df94ee019,network.aragonid.eth,Aragon Network,None


In [97]:
dao_names['mainnet'] = dfn_concat.to_dict('records')
dao_names

{'mainnet': [{'address': '0xf47917b108ca4b820ccea2587546fbb9f7564b56',
   'domain': 'dcl.eth',
   'name': 'Decentraland',
   'source': None},
  {'address': '0xfe1f2de598f42ce67bb9aad5ad473f0272d09b74',
   'domain': 'meloncouncil.eth',
   'name': 'Melon Council',
   'source': None},
  {'address': '0x2de83b50af29678774d5abc4a7cb2a588762f28c',
   'domain': 'governance.aragonproject.eth',
   'name': 'Aragon Governance',
   'source': None},
  {'address': '0x635193983512c621e6a3e15ee1dbf36f0c0db8e0',
   'domain': 'a1.aragonid.eth',
   'name': 'Aragon One',
   'source': None},
  {'address': '0x67757a18eda83125270ef94dcec7658eb39bd8a5',
   'domain': 'blankdao.aragonid.eth',
   'name': 'BlankDAO',
   'source': None},
  {'address': '0xcd3d9b832bff15e0a519610372c6aac651872dde',
   'domain': '',
   'name': 'MyBit',
   'source': None},
  {'address': '0x0ee165029b09d91a54687041adbc705f6376c67f',
   'domain': '',
   'name': 'Livepeer',
   'source': None},
  {'address': '0x5aad137d8f7d2dc6e1b2548c059b

In [98]:
with open('../../cache_scripts/aragon/dao_names.json', 'w') as jf:
    json.dump(dao_names, jf, indent=2)